In [ ]:
import sys
from pathlib import Path

from astropy.table import Table
from matplotlib import pyplot as plt
from sndata.sdss import sako18spec

sys.path.insert(0, '../')
from phot_class.spectra import tabulate_spectral_properties


In [ ]:
sako18spec.download_module_data()
results_dir = Path('/Users/daniel/Github/Photometric-Classification/results/')


In [ ]:
spec_class = Table.read(results_dir / 'spec_class/sdss_sako18spec.ecsv').to_pandas()
spec_class.set_index(['feat_name', 'obj_id', 'date'], inplace=True)

spec_summary = sako18spec.load_table(9).to_pandas()
spec_summary = spec_summary[spec_summary['Type'] != 'Gal']
spec_summary.drop_duplicates('CID', inplace=True)

spec_summary.rename(columns={'CID': 'obj_id', 'Date': 'date'}, inplace=True)
spec_summary.set_index(['obj_id'], inplace=True)

spec_data = spec_class.join(spec_summary)
spec_data


In [ ]:
ia_data = spec_data[spec_data.Type.isin(['Ia', 'Ia-pec', 'Ia?'])]

for sdss_class, spectral_measurements in ia_data.groupby('Type'):
    si_features = spectral_measurements.pew.loc[['pW6', 'pW7']]
    good_data = (~si_features.isna()).index
    plot_data = si_features.reindex(good_data)
    
    y = plot_data.loc['pW6']
    x = plot_data.loc['pW7']
    
    plt.scatter(x, y, label=sdss_class)

plt.xlim(-100, 800)
plt.ylim(-60, 300)
plt.legend()
